# One-card poker game with private information

In this tutorial, we'll create an extensive form representation of a one-card poker game ([Mye91](#mye91)) and use it to demonstrate and explain the following with Gambit:

1. Setting up an extensive form game with imperfect information
2. Using information sets
3. [Retrieving player payoff tables from the game](#)
4. [Computing Nash equilibria](#)
5. [Acceptance criteria for Nash equilibria](#)

A version of this game also appears in [RUW08](#ruw08), as a classroom game under the name "stripped-down poker".

This is perhaps the simplest interesting game with imperfect information.

In our version of the game, there are two players, **Alice** and **Bob**, and a deck of cards, with equal numbers of **King** and **Queen** cards.

- The game begins with each player putting $1 in the pot.
- A card is dealt at random to Alice
    - Alice observes her card
    - Bob does not observe the card
- Alice then chooses either to **Raise** or to **Fold**.
    - If she chooses to Fold, Bob wins the pot and the game ends.
    - If she chooses to Raise, she adds another $1 to the pot.
- Bob then chooses either to **Meet** or **Pass**.
    - If he chooses to Pass, Alice wins the pot and the game ends.
    - If he chooses to Meet, he adds another $1 to the pot.
- There is then a showdown, in which Alice reveals her card.
    - If she has a King, then she wins the pot;
    - If she has a Queen, then Bob wins the pot.

In [63]:
import pygambit as gbt

Create the game with two players.

In [64]:
g = gbt.Game.new_tree(
    players=["Alice", "Bob"], 
    title="One card poker"
)

In addition to the two named players, Gambit also instantiates a chance player.

In [65]:
print(g.players["Alice"])
print(g.players["Bob"])
print(g.players.chance)

Player(game=Game(title='One card poker'), label='Alice')
Player(game=Game(title='One card poker'), label='Bob')
ChancePlayer(game=Game(title='One card poker'))


Moves belonging to the chance player can be added in the same way as to other players.

At any new move created for the chance player, the action probabilities default to uniform randomization over the actions at the move.

The first step in this game is that Alice is dealt a card which could be a King or Queen, each with probability 1/2.

To simulate this in Gambit, we create a chance player move at the root node of the game.

In [66]:
g.append_move(
    g.root,
    player=g.players.chance,
    actions=["King", "Queen"]  # By default, chance actions have equal probabilities
)
g.root.children[0].label = "King"  # TODO: Update API such that labels are set during move creation
g.root.children[1].label = "Queen"

Now let's add Alice's first move after the card is dealt.

In this game, information structure is important.
Alice knows her card, so the two nodes at which she has the move are part of different information sets.

We'll therefore need to append Alice's move separately for each of the root node's children, i.e. the scenarios where she has a King or a Queen.

In [67]:
for node in g.root.children:
    g.append_move(
        node,
        player="Alice",
        actions=["Raise", "Fold"]
    )
    node.children[0].label = "Raise"  # TODO: Update API such that labels are set during move creation
    node.children[1].label = "Fold"

The loop above causes each of the newly-appended moves to be in new **information sets**, reflecting the fact that Alice's decision depends on the knowledge of which card she holds.

In contrast, Bob does not know Alice’s card, and therefore cannot distinguish between the two nodes at which he has to make his decision:

  - Chance player chooses King, then Alice Raises: `g.root.children["King"].children["Raise"]`
  - Chance player chooses Queen, then Alice Raises: `g.root.children["Queen"].children["Raise"`

In other words, Bob's decision when Alice has a Queen should be part of the same information set as Bob's decision when Alice has a King.

To set this scenario up in Gambit, we'll need to use `Game.append_infoset` to add a move as part of an existing information set (represented in Gambit as an `Infoset`).

First, let's add Bob's move to the node where Alice has raised with a King.

In [68]:
g.append_move(
    g.root.children["King"].children["Raise"],
    player="Bob",
    actions=["Meet", "Pass"]
)
g.root.children["King"].children["Raise"].children[0].label = "Meet"  # TODO: Update API such that labels are set during move creation
g.root.children["King"].children["Raise"].children[1].label = "Pass"

Now let's add the information set we created at the node where Alice raised with a King, to the node where Alice raised with a Queen.

In [69]:
g.append_infoset(
    g.root.children["Queen"].children["Raise"],
    infoset=g.root.children["King"].children["Raise"].infoset
)
g.root.children["Queen"].children["Raise"].children[0].label = "Meet"  # TODO: Update API such that labels are set during move creation
g.root.children["Queen"].children["Raise"].children[1].label = "Pass"

In game theory terms, this creates "imperfect information".
Bob cannot distinguish between these two nodes in the game tree, so he must use the same strategy (same probabilities for Meet vs. Pass) in both situations.

This is crucial in games where players must make decisions without complete knowledge of their opponents' private information.

Let's now set up the four possible payoff outcomes for the game.

In [70]:
alice_winsbig = g.add_outcome([2, -2], label="Alice wins big")
alice_wins = g.add_outcome([1, -1], label="Alice wins")
bob_winsbig = g.add_outcome([-2, 2], label="Bob wins big")
bob_wins = g.add_outcome([-1, 1], label="Bob wins")

Finally, we should assign an outcome to each of the terminal nodes in the game tree.

In [71]:
# Alice folds, Bob wins small
g.set_outcome(g.root.children["King"].children["Fold"], bob_wins)
g.set_outcome(g.root.children["Queen"].children["Fold"], bob_wins)

# Bob sees Alice raise and calls, correctly believing she is bluffing, Bob wins big
g.set_outcome(g.root.children["Queen"].children["Raise"].children["Meet"], bob_winsbig)

# Bob sees Alice raise and calls, incorrectly believing she is bluffing, Alice wins big
g.set_outcome(g.root.children["King"].children["Raise"].children["Meet"], alice_winsbig)

# Bob does not call Alice's raise, Alice wins small
g.set_outcome(g.root.children["King"].children["Raise"].children["Pass"], alice_wins)
g.set_outcome(g.root.children["Queen"].children["Raise"].children["Pass"], alice_wins)

In [72]:
# m, m_transposed = g.to_arrays()
# print(m)
# print(m_transposed)